In [1]:
# Databricks notebook source
import os
import re
import gc
import glob
import pickle
import warnings
import os.path
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
import pandas.io.common
from scipy import stats
from datetime import datetime
from scipy.special import inv_boxcox
# from catboost.utils import eval_metric
# from catboost import CatBoostRegressor,Pool
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [6]:
loc_path = r'/Users/raj2.gaurav/Desktop/' 
ADS_features_path = loc_path + 'Causal Model/05. Training Data/'
df = pd.read_csv(ADS_features_path + 'ADS_Bihar_Nov.csv')

In [7]:
df.head()

,Circle,City,Component,Partition_date,Hour,Total customers,Anomaly,Ratio data,Ratio voice,Ratio hsi,Cluster_issue,Data volume,Data volume formula,Data volume mean,Data volume std,Voice affected users,Voice affected users formula,Voice affected users mean,Voice affected users std,Hsi affected users,Hsi affected users formula,Hsi affected users mean,Hsi affected users std,Num_grids,Num_cells,Num_sectors,Indoor frqeqency % 850,Indoor frqeqency % 1800,Indoor frqeqency % 2300,Prb_10,Prb_10_25,Prb_25_70,Prb_70_90,Prb_90,Target_Cluster,Sum_total_volume_mb_download,Sum_total_volume_mb_upload,Unique_customers,Total_duration_sec,Total_sessions,Rsrp_nom,Rsrp_dnom,Rsrq_nom,Rsrq_dnom,Mean_cqi_nom,Mean_cqi_dnom,Pusch_sinr_nom,Pusch_sinr_dnom,Ta_distance_nom,Ta_distance_dnom,Intersite_distance_nom,Intersite_distance_dnom,Rsrp_nom_lag_1_day,Rsrp_dnom_lag_1_day,Rsrq_nom_lag_1_day,Rsrq_dnom_lag_1_day,Avg_bad_hsi_pct,Avg_bad_voice_pct,Ph_mean_prb_dl,Totprbdlmin_perc,Totprbdlmax_perc,Totprbdltot_perc,Totprbdlcnt_cnt
0,bihar,deoghar,33,2022-11-06,5,4726,Voice,1,4,2,NaN,27,49,16,10,267,223,62,53,65,223,25,66,9,35,10,31,31,36,9.00,22.00,4.00,0.00,0.00,0,12416679610.00,849541683.00,1393.00,457966.00,16223.00,-852588.00,8767.00,-93934.00,8767.00,172773.00,16154.00,243832.40,15323.00,6892183.00,16223.00,8839869.00,16223.00,-852588.00,8767.00,-93934.00,8767.00,9.84,19.86,11.67,2.08,49.56,1360.46,173.61
1,bihar,deoghar,33,2022-11-06,6,6551,Voice,1,3,2,NaN,47,85,27,19,499,495,144,116,160,447,73,124,9,35,10,35,31,33,6.00,19.00,10.00,0.00,0.00,0,22920760951.00,2194595046.00,1808.00,651638.00,20006.00,-1077896.00,11200.00,-127212.00,11200.00,206115.00,19937.00,287435.70,18246.00,7775528.00,20006.00,10909146.00,20006.00,-1077896.00,11200.00,-127212.00,11200.00,9.34,21.39,23.32,2.92,69.31,2859.01,175.75
2,bihar,katihar,116,2022-11-06,11,13276,HSI,2,2,4,high_prb,137,191,46,48,888,1478,352,375,1533,1532,337,398,8,27,8,32,39,27,0.00,0.00,8.00,8.00,11.00,1,38271955206.00,3384246507.00,2182.00,500347.00,25844.00,-1520924.00,15271.00,-213869.00,15271.00,247706.00,25724.00,118385.60,8936.00,16869064.00,25844.00,44695061.00,25844.00,-1520924.00,15271.00,-213869.00,15271.00,0.94,43.54,48.62,11.19,93.67,7589.50,175.89
3,bihar,katihar,116,2022-11-06,12,14047,HSI,2,2,4,high_prb,141,197,47,49,851,1454,342,370,1656,1616,351,421,8,27,8,33,40,26,0.00,0.00,7.00,10.00,10.00,1,43646120543.00,3608188983.00,2094.00,480070.00,26834.00,-1679304.00,16734.00,-232907.00,16734.00,252646.00,26701.00,110929.50,8704.00,17197404.00,26834.00,44264376.00,26834.00,-1679304.00,16734.00,-232907.00,16734.00,7.23,11.55,57.34,11.84,96.62,8736.38,175.75
4,bihar,patna,85,2022-11-06,7,11028,Data - Voice,4,3,0,NaN,166,129,35,31,991,887,252,211,113,674,131,180,13,60,19,30,31,38,0.00,11.00,41.00,5.00,3.00,0,49107207869.00,4211397132.00,3595.00,1627912.00,50063.00,-2857620.00,29925.00,-379860.50,29925.00,458375.00,49864.00,393226.90,35822.00,24208274.00,50063.00,35345652.00,50063.00,-2857620.00,29925.00,-379860.50,29925.00,10.52,14.38,45.71,9.04,83.81,6210.03,175.75


In [4]:
df.shape

(33310, 60)

In [8]:
df.columns

Index(['Circle', 'City', 'Component', 'Partition_date', 'Hour',
       'Total customers', 'Anomaly', 'Ratio data', 'Ratio voice', 'Ratio hsi',
       'Cluster_issue', 'Data volume', 'Data volume formula',
       'Data volume mean', 'Data volume std', 'Voice affected users',
       'Voice affected users formula', 'Voice affected users mean',
       'Voice affected users std', 'Hsi affected users',
       'Hsi affected users formula', 'Hsi affected users mean',
       'Hsi affected users std', 'Num_grids', 'Num_cells', 'Num_sectors',
       'Indoor frqeqency % 850', 'Indoor frqeqency % 1800',
       'Indoor frqeqency % 2300', 'Prb_10', 'Prb_10_25', 'Prb_25_70',
       'Prb_70_90', 'Prb_90', 'Target_Cluster', 'Sum_total_volume_mb_download',
       'Sum_total_volume_mb_upload', 'Unique_customers', 'Total_duration_sec',
       'Total_sessions', 'Rsrp_nom', 'Rsrp_dnom', 'Rsrq_nom', 'Rsrq_dnom',
       'Mean_cqi_nom', 'Mean_cqi_dnom', 'Pusch_sinr_nom', 'Pusch_sinr_dnom',
       'Ta_distance_n

In [9]:
df['Ratio data'].unique()

array([ 1,  2,  4,  5,  6,  3,  7, 10, 11, 13,  9, 12,  0,  8, 14, 15, 16,
       17, 19, 20, 21, 18])

In [10]:
df['Target_Cluster'].value_counts()

0    19161
1    14149
Name: Target_Cluster, dtype: int64

In [12]:
df.drop(['Anomaly','Cluster_issue','Unique_customers'],axis=1,inplace=True)

In [13]:
df.head()

,Circle,City,Component,Partition_date,Hour,Total customers,Ratio data,Ratio voice,Ratio hsi,Data volume,Data volume formula,Data volume mean,Data volume std,Voice affected users,Voice affected users formula,Voice affected users mean,Voice affected users std,Hsi affected users,Hsi affected users formula,Hsi affected users mean,Hsi affected users std,Num_grids,Num_cells,Num_sectors,Indoor frqeqency % 850,Indoor frqeqency % 1800,Indoor frqeqency % 2300,Prb_10,Prb_10_25,Prb_25_70,Prb_70_90,Prb_90,Target_Cluster,Sum_total_volume_mb_download,Sum_total_volume_mb_upload,Total_duration_sec,Total_sessions,Rsrp_nom,Rsrp_dnom,Rsrq_nom,Rsrq_dnom,Mean_cqi_nom,Mean_cqi_dnom,Pusch_sinr_nom,Pusch_sinr_dnom,Ta_distance_nom,Ta_distance_dnom,Intersite_distance_nom,Intersite_distance_dnom,Rsrp_nom_lag_1_day,Rsrp_dnom_lag_1_day,Rsrq_nom_lag_1_day,Rsrq_dnom_lag_1_day,Avg_bad_hsi_pct,Avg_bad_voice_pct,Ph_mean_prb_dl,Totprbdlmin_perc,Totprbdlmax_perc,Totprbdltot_perc,Totprbdlcnt_cnt
0,bihar,deoghar,33,2022-11-06,5,4726,1,4,2,27,49,16,10,267,223,62,53,65,223,25,66,9,35,10,31,31,36,9.00,22.00,4.00,0.00,0.00,0,12416679610.00,849541683.00,457966.00,16223.00,-852588.00,8767.00,-93934.00,8767.00,172773.00,16154.00,243832.40,15323.00,6892183.00,16223.00,8839869.00,16223.00,-852588.00,8767.00,-93934.00,8767.00,9.84,19.86,11.67,2.08,49.56,1360.46,173.61
1,bihar,deoghar,33,2022-11-06,6,6551,1,3,2,47,85,27,19,499,495,144,116,160,447,73,124,9,35,10,35,31,33,6.00,19.00,10.00,0.00,0.00,0,22920760951.00,2194595046.00,651638.00,20006.00,-1077896.00,11200.00,-127212.00,11200.00,206115.00,19937.00,287435.70,18246.00,7775528.00,20006.00,10909146.00,20006.00,-1077896.00,11200.00,-127212.00,11200.00,9.34,21.39,23.32,2.92,69.31,2859.01,175.75
2,bihar,katihar,116,2022-11-06,11,13276,2,2,4,137,191,46,48,888,1478,352,375,1533,1532,337,398,8,27,8,32,39,27,0.00,0.00,8.00,8.00,11.00,1,38271955206.00,3384246507.00,500347.00,25844.00,-1520924.00,15271.00,-213869.00,15271.00,247706.00,25724.00,118385.60,8936.00,16869064.00,25844.00,44695061.00,25844.00,-1520924.00,15271.00,-213869.00,15271.00,0.94,43.54,48.62,11.19,93.67,7589.50,175.89
3,bihar,katihar,116,2022-11-06,12,14047,2,2,4,141,197,47,49,851,1454,342,370,1656,1616,351,421,8,27,8,33,40,26,0.00,0.00,7.00,10.00,10.00,1,43646120543.00,3608188983.00,480070.00,26834.00,-1679304.00,16734.00,-232907.00,16734.00,252646.00,26701.00,110929.50,8704.00,17197404.00,26834.00,44264376.00,26834.00,-1679304.00,16734.00,-232907.00,16734.00,7.23,11.55,57.34,11.84,96.62,8736.38,175.75
4,bihar,patna,85,2022-11-06,7,11028,4,3,0,166,129,35,31,991,887,252,211,113,674,131,180,13,60,19,30,31,38,0.00,11.00,41.00,5.00,3.00,0,49107207869.00,4211397132.00,1627912.00,50063.00,-2857620.00,29925.00,-379860.50,29925.00,458375.00,49864.00,393226.90,35822.00,24208274.00,50063.00,35345652.00,50063.00,-2857620.00,29925.00,-379860.50,29925.00,10.52,14.38,45.71,9.04,83.81,6210.03,175.75


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33310 entries, 0 to 33309
Data columns (total 60 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Circle                        33310 non-null  object 
 1   City                          33310 non-null  object 
 2   Component                     33310 non-null  int64  
 3   Partition_date                33310 non-null  object 
 4   Hour                          33310 non-null  int64  
 5   Total customers               33310 non-null  int64  
 6   Ratio data                    33310 non-null  int64  
 7   Ratio voice                   33310 non-null  int64  
 8   Ratio hsi                     33310 non-null  int64  
 9   Data volume                   33310 non-null  int64  
 10  Data volume formula           33310 non-null  int64  
 11  Data volume mean              33310 non-null  int64  
 12  Data volume std               33310 non-null  int64  
 13  V

In [15]:
df.describe()

,Component,Hour,Total customers,Ratio data,Ratio voice,Ratio hsi,Data volume,Data volume formula,Data volume mean,Data volume std,Voice affected users,Voice affected users formula,Voice affected users mean,Voice affected users std,Hsi affected users,Hsi affected users formula,Hsi affected users mean,Hsi affected users std,Num_grids,Num_cells,Num_sectors,Indoor frqeqency % 850,Indoor frqeqency % 1800,Indoor frqeqency % 2300,Prb_10,Prb_10_25,Prb_25_70,Prb_70_90,Prb_90,Target_Cluster,Sum_total_volume_mb_download,Sum_total_volume_mb_upload,Total_duration_sec,Total_sessions,Rsrp_nom,Rsrp_dnom,Rsrq_nom,Rsrq_dnom,Mean_cqi_nom,Mean_cqi_dnom,Pusch_sinr_nom,Pusch_sinr_dnom,Ta_distance_nom,Ta_distance_dnom,Intersite_distance_nom,Intersite_distance_dnom,Rsrp_nom_lag_1_day,Rsrp_dnom_lag_1_day,Rsrq_nom_lag_1_day,Rsrq_dnom_lag_1_day,Avg_bad_hsi_pct,Avg_bad_voice_pct,Ph_mean_prb_dl,Totprbdlmin_perc,Totprbdlmax_perc,Totprbdltot_perc,Totprbdlcnt_cnt
count,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,33310.00,28497.00,28497.00,28497.00,28497.00,28497.00,28497.00,28497.00,28497.00,28497.00,28497.00,28480.00,28497.00,28497.00,28497.00,28497.00,28497.00,24788.00,24788.00,24788.00,24788.00,28688.00,29556.00,28482.00,28482.00,28482.00,28482.00,28482.00
mean,109.90,11.85,15205.06,4.31,4.56,6.06,184.11,155.13,36.85,38.87,1311.59,1080.49,257.26,273.89,1078.45,1041.92,204.68,278.58,13.33,59.50,17.64,30.87,34.71,32.92,5.25,9.03,27.27,11.37,6.50,0.42,93473990596.14,8873669455.80,3005602.45,84727.97,-5377700.20,57395.28,-791063.75,57395.28,724408.99,84187.39,676381.38,51734.30,32674345.47,84714.40,44887646.66,84714.65,-5406679.71,57712.98,-795426.34,57712.98,11.79,18.04,46.09,11.93,81.26,6796.74,170.87
std,77.69,6.52,9328.48,2.72,2.69,8.81,123.33,56.47,12.62,14.84,993.67,577.01,137.39,147.08,1009.10,582.00,135.94,151.03,7.38,31.74,9.51,4.64,6.22,7.52,12.53,11.21,20.28,10.12,7.67,0.49,119321299491.03,11171883453.94,3522903.18,88774.67,5833959.51,63412.40,898674.72,63412.40,745657.61,88262.83,706874.43,53605.25,27484646.17,88758.19,38289363.17,88758.22,5842000.62,63506.35,899955.74,63506.35,8.55,5.71,20.23,8.70,18.77,3374.64,15.70
min,0.00,0.00,363.00,0.00,0.00,0.00,0.00,33.00,8.00,7.00,2.00,54.00,13.00,13.00,1.00,73.00,6.00,22.00,1.00,3.00,1.00,4.00,6.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,39357563.00,8795501.00,0.00,188.00,-39332548.00,56.00,-6031618.00,56.00,2199.00,186.00,2779.20,0.00,184475.00,188.00,362952.00,188.00,-39332548.00,56.00,-6031618.00,56.00,0.00,0.38,2.25,1.26,7.89,319.32,58.58
25%,45.00,6.00,8641.00,2.00,3.00,2.00,75.00,129.00,30.00,31.00,450.00,531.00,138.00,126.00,318.00,490.00,73.00,137.00,7.00,31.00,9.00,29.00,32.00,31.00,0.00,2.00,8.00,3.00,1.00,0.00,20564863326.00,1861083017.00,656542.00,21285.00,-7182315.00,14732.00,-1063233.50,14732.00,183411.00,21158.00,158865.50,13344.00,14552620.00,21285.00,19664180.00,21285.00,-7265684.00,14881.00,-1071375.00,14881.00,6.44,14.58,33.57,5.45,77.61,4494.65,175.75
50%,99.00,12.00,13863.00,4.00,4.00,4.00,178.00,175.00,41.00,44.00,1192.00,1390.00,334.00,352.00,882.00,1194.00,224.00,315.00,14.00,63.00,18.00,31.00,34.00,34.00,1.00,5.00,25.00,9.00,4.00,0.00,45822401910.00,4318335393.00,1536602.00,48423.00,-2954305.00,30966.00,-415171.00,30966.00,410798.00,48078.00,370463.90,28393.00,24357813.00,48423.00,34043662.00,48423.00,-2990123.00,31233.00,-419509.00,31233.00,9.40,17.16,47.37,10.22,88.51,6914.98,175.75
75%,178.00,17.00,20383.50,6.00,6.00,7.00,263.00,192.00,45.00,48.00,1957.00,1559.00,366.00,397.00,1569.00,1479.00,300.00,389.00,18.00,83.00,25.00,33.00,37.00,36.00,3.00,11.00,42.00,18.00,9.00,1.00,120456496486.00,11465796665.00,3927768.00,119803.00,-1440745.00,76971.00,-195212.50,76971.00,1057441.00,119005.00,1002833.02,75030.00,42130887.00,119803.00,56963733.00,119803.00,-1454664.25,77576.0

In [19]:
df.sort_values('Partition_date',inplace=True)

In [20]:
df.head()

,Circle,City,Component,Partition_date,Hour,Total customers,Ratio data,Ratio voice,Ratio hsi,Data volume,Data volume formula,Data volume mean,Data volume std,Voice affected users,Voice affected users formula,Voice affected users mean,Voice affected users std,Hsi affected users,Hsi affected users formula,Hsi affected users mean,Hsi affected users std,Num_grids,Num_cells,Num_sectors,Indoor frqeqency % 850,Indoor frqeqency % 1800,Indoor frqeqency % 2300,Prb_10,Prb_10_25,Prb_25_70,Prb_70_90,Prb_90,Target_Cluster,Sum_total_volume_mb_download,Sum_total_volume_mb_upload,Total_duration_sec,Total_sessions,Rsrp_nom,Rsrp_dnom,Rsrq_nom,Rsrq_dnom,Mean_cqi_nom,Mean_cqi_dnom,Pusch_sinr_nom,Pusch_sinr_dnom,Ta_distance_nom,Ta_distance_dnom,Intersite_distance_nom,Intersite_distance_dnom,Rsrp_nom_lag_1_day,Rsrp_dnom_lag_1_day,Rsrq_nom_lag_1_day,Rsrq_dnom_lag_1_day,Avg_bad_hsi_pct,Avg_bad_voice_pct,Ph_mean_prb_dl,Totprbdlmin_perc,Totprbdlmax_perc,Totprbdltot_perc,Totprbdlcnt_cnt
33309,bihar,sitamarhi,63,2022-11-03,5,1552,0,0,9,9,48,16,10,17,227,64,54,282,246,30,72,2,6,2,44,38,17,0.00,1.00,3.00,0.00,2.00,1,9871389937.00,1654724112.00,557714.00,23044.00,-942453.00,10461.00,-107695.00,10461.00,230431.00,22922.00,317421.60,22027.00,7944067.00,23044.00,18059869.00,23044.00,-942453.00,10461.00,-107695.00,10461.00,5.21,18.02,18.14,2.53,61.52,2196.04,173.95
30382,bihar,patna,63,2022-11-03,7,13152,3,3,0,127,122,34,29,1064,944,266,225,129,674,132,180,18,71,23,33,34,32,7.00,25.00,39.00,0.00,0.00,0,9856893226.00,862261495.00,381528.00,12052.00,-1081916.00,10373.00,-144243.50,10373.00,84023.00,11879.00,65251.80,9105.00,19772113.00,12052.00,18760146.00,12052.00,-1081916.00,10373.00,-144243.50,10373.00,13.49,19.96,47.84,6.82,88.18,6352.62,175.75
30381,bihar,patna,63,2022-11-03,1,6558,4,5,1,62,76,14,20,110,90,19,23,15,77,7,23,18,71,23,29,32,38,28.00,32.00,11.00,0.00,0.00,1,2935410678.00,237940740.00,182219.00,7346.00,-498656.00,4789.00,-51191.00,4789.00,74894.00,7310.00,77956.20,6900.00,8466981.00,7346.00,11477424.00,7346.00,-498656.00,4789.00,-51191.00,4789.00,NaN,26.30,10.94,2.26,53.40,1332.00,175.75
30380,bihar,patna,63,2022-11-03,13,19699,5,5,1,232,179,44,44,1919,1513,355,386,636,1479,333,382,18,71,23,32,34,32,1.00,8.00,50.00,11.00,1.00,0,10146439674.00,857247531.00,381101.00,12850.00,-1108100.00,10877.00,-150361.00,10877.00,98723.00,12725.00,100631.20,8871.00,18187110.00,12850.00,19918748.00,12850.00,-1108100.00,10877.00,-150361.00,10877.00,12.51,27.55,52.66,8.08,89.47,6967.67,175.75
30379,bihar,patna,63,2022-11-03,14,20045,5,5,2,231,188,45,47,1865,1465,345,373,684,1470,328,380,18,71,23,34,33,32,1.00,9.00,44.00,16.00,1.00,0,12685823192.00,1078333886.00,404007.00,12746.00,-1043080.00,10211.00,-141345.50,10211.00,101052.00,12643.00,104754.60,8898.00,18058176.00,12746.00,19749740.00,12746.00,-1043080.00,10211.00,-141345.50,10211.00,17.68,26.00,45.01,7.72,88.94,6180.85,175.75


In [21]:
df.isnull().sum()

Circle                             0
City                               0
Component                          0
Partition_date                     0
Hour                               0
Total customers                    0
Ratio data                         0
Ratio voice                        0
Ratio hsi                          0
Data volume                        0
Data volume formula                0
Data volume mean                   0
Data volume std                    0
Voice affected users               0
Voice affected users formula       0
Voice affected users mean          0
Voice affected users std           0
Hsi affected users                 0
Hsi affected users formula         0
Hsi affected users mean            0
Hsi affected users std             0
Num_grids                          0
Num_cells                          0
Num_sectors                        0
Indoor frqeqency % 850             0
Indoor frqeqency % 1800            0
Indoor frqeqency % 2300            0
P

In [22]:
df.columns.values

array(['Circle', 'City', 'Component', 'Partition_date', 'Hour',
       'Total customers', 'Ratio data', 'Ratio voice', 'Ratio hsi',
       'Data volume', 'Data volume formula', 'Data volume mean',
       'Data volume std', 'Voice affected users',
       'Voice affected users formula', 'Voice affected users mean',
       'Voice affected users std', 'Hsi affected users',
       'Hsi affected users formula', 'Hsi affected users mean',
       'Hsi affected users std', 'Num_grids', 'Num_cells', 'Num_sectors',
       'Indoor frqeqency % 850', 'Indoor frqeqency % 1800',
       'Indoor frqeqency % 2300', 'Prb_10', 'Prb_10_25', 'Prb_25_70',
       'Prb_70_90', 'Prb_90', 'Target_Cluster',
       'Sum_total_volume_mb_download', 'Sum_total_volume_mb_upload',
       'Total_duration_sec', 'Total_sessions', 'Rsrp_nom', 'Rsrp_dnom',
       'Rsrq_nom', 'Rsrq_dnom', 'Mean_cqi_nom', 'Mean_cqi_dnom',
       'Pusch_sinr_nom', 'Pusch_sinr_dnom', 'Ta_distance_nom',
       'Ta_distance_dnom', 'Intersite_dista

In [23]:
fill_val_lst=['Sum_total_volume_mb_download', 'Sum_total_volume_mb_upload',
       'Total_duration_sec', 'Total_sessions', 'Rsrp_nom', 'Rsrp_dnom',
       'Rsrq_nom', 'Rsrq_dnom', 'Mean_cqi_nom', 'Mean_cqi_dnom',
       'Pusch_sinr_nom', 'Pusch_sinr_dnom', 'Ta_distance_nom',
       'Ta_distance_dnom', 'Intersite_distance_nom',
       'Intersite_distance_dnom', 'Rsrp_nom_lag_1_day',
       'Rsrp_dnom_lag_1_day', 'Rsrq_nom_lag_1_day', 'Rsrq_dnom_lag_1_day',
       'Avg_bad_hsi_pct', 'Avg_bad_voice_pct', 'Ph_mean_prb_dl',
       'Totprbdlmin_perc', 'Totprbdlmax_perc', 'Totprbdltot_perc',
       'Totprbdlcnt_cnt']

In [24]:
for i in fill_val_lst:
    df[i].fillna(df[i].mean(),inplace=True)

### Modelling

In [31]:
df_model=df.copy()

In [32]:
df_model.dtypes

Circle                           object
City                             object
Component                         int64
Partition_date                   object
Hour                              int64
Total customers                   int64
Ratio data                        int64
Ratio voice                       int64
Ratio hsi                         int64
Data volume                       int64
Data volume formula               int64
Data volume mean                  int64
Data volume std                   int64
Voice affected users              int64
Voice affected users formula      int64
Voice affected users mean         int64
Voice affected users std          int64
Hsi affected users                int64
Hsi affected users formula        int64
Hsi affected users mean           int64
Hsi affected users std            int64
Num_grids                         int64
Num_cells                         int64
Num_sectors                       int64
Indoor frqeqency % 850            int64


In [33]:
encoding_lst=['City','Partition_date']

In [34]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()

In [35]:
for i in range(0, len(encoding_lst)):
    df[f'{encoding_lst[i]}'] = pd.Categorical(df[f'{encoding_lst[i]}'])
    dummies = pd.get_dummies(df[f'{encoding_lst[i]}'], prefix = f'{encoding_lst[i]}_encoded')
    df.drop([f'{encoding_lst[i]}'], axis=1, inplace=True)
    df = pd.concat([df, dummies], axis=1)

In [39]:
X=df.drop(['Circle','Target_Cluster'],axis=1)
y=df['Target_Cluster']

In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [75]:
X_test.head()

,Component,Hour,Total customers,Ratio data,Ratio voice,Ratio hsi,Data volume,Data volume formula,Data volume mean,Data volume std,Voice affected users,Voice affected users formula,Voice affected users mean,Voice affected users std,Hsi affected users,Hsi affected users formula,Hsi affected users mean,Hsi affected users std,Num_grids,Num_cells,Num_sectors,Indoor frqeqency % 850,Indoor frqeqency % 1800,Indoor frqeqency % 2300,Prb_10,Prb_10_25,Prb_25_70,Prb_70_90,Prb_90,Sum_total_volume_mb_download,Sum_total_volume_mb_upload,Total_duration_sec,Total_sessions,Rsrp_nom,Rsrp_dnom,Rsrq_nom,Rsrq_dnom,Mean_cqi_nom,Mean_cqi_dnom,Pusch_sinr_nom,Pusch_sinr_dnom,Ta_distance_nom,Ta_distance_dnom,Intersite_distance_nom,Intersite_distance_dnom,Rsrp_nom_lag_1_day,Rsrp_dnom_lag_1_day,Rsrq_nom_lag_1_day,Rsrq_dnom_lag_1_day,Avg_bad_hsi_pct,Avg_bad_voice_pct,Ph_mean_prb_dl,Totprbdlmin_perc,Totprbdlmax_perc,Totprbdltot_perc,Totprbdlcnt_cnt,City_encoded_araria,City_encoded_arrah,City_encoded_bagaha,City_encoded_barauni,City_encoded_begusarai,City_encoded_bettiah,City_encoded_bhagalpur,City_encoded_buxar,City_encoded_chaibasa,City_encoded_chakradharpur,City_encoded_chapra,City_encoded_chirkunda,City_encoded_daltonganj,City_encoded_darbhanga,City_encoded_dehri,City_encoded_deoghar,City_encoded_dhanbad,City_encoded_dumka,City_encoded_gaya,City_encoded_giridih,City_encoded_gopalganj,City_encoded_hajipur,City_encoded_hazaribag,City_encoded_jamalpur,City_encoded_jamshedpur,City_encoded_jamui,City_encoded_jehanabad,City_encoded_katihar,City_encoded_khagaria,City_encoded_kishanganj,City_encoded_lakhisarai,City_encoded_madhubani,City_encoded_mokameh,City_encoded_motihari,City_encoded_munger,City_encoded_muzaffarpur,City_encoded_nawada,City_encoded_patna,City_encoded_phusro,City_encoded_purnia,City_encoded_rajgir,City_encoded_ramgarh,City_encoded_ranchi,City_encoded_saharsa,City_encoded_sahibganj,City_encoded_samastipur,City_encoded_sasaram,City_encoded_saunda,City_encoded_sitamarhi,City_encoded_siwan,City_encoded_supaul,Partition_date_encoded_2022-11-03,Partition_date_encoded_2022-11-04,Partition_date_encoded_2022-11-05,Partition_date_encoded_2022-11-06,Partition_date_encoded_2022-11-07,Partition_date_encoded_2022-11-08,Partition_date_encoded_2022-11-09
12664,105,14,21933,7,7,2,352,199,46,50,2528,1508,346,387,777,1422,261,387,24,97,30,29,34,36,3.00,14.00,54.00,21.00,5.00,45553711483.00,4388696787.00,1415960.00,52673.00,-4399261.00,44785.00,-672003.50,44785.00,373440.00,52048.00,344675.70,28251.00,54015634.00,52645.00,61437694.00,52673.00,-4399261.00,44785.00,-672003.50,44785.00,9.34,14.71,47.14,12.60,91.51,7229.54,175.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
24580,225,11,27446,7,7,3,323,176,43,44,2757,1583,370,404,907,1314,285,342,23,96,29,29,37,33,5.00,13.00,57.00,18.00,3.00,166481345037.00,19026618756.00,7165243.00,227140.00,-15012762.00,160541.00,-2262695.00,160541.00,1841243.00,225460.00,1577741.00,125619.00,45939872.00,227140.00,64715243.00,227140.00,-15012762.00,160541.00,-2262695.00,160541.00,10.04,20.06,48.63,11.54,88.31,7200.98,175.81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
20035,2,9,20871,6,6,3,258,154,38,38,2223,1437,364,357,878,1053,228,275,17,78,23,33,35,31,0.00,8.00,42.00,23.00,5.00,93473990596.14,8873669455.80,3005602.45,84727.97,-5377700.20,57395.28,-791063.75,57395.28,724408.99,84187.39,676381.38,51734.30,32674345.47,84714.40,44887646.66,84714.65,-5406679.71,57712.98,-795426.34,57712.98,11.79,18.04,46.09,11.93,81.26,6796.74,170.87,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
11656,162,20,16718,4,4,2,219,206,51,51,1557,1451,324,375,928,1766,399,455,14,59,17,30,33,35,1.00,8.00,29.00,15.00,6.00,80358550231.00,6527326309.00,2408862.00,56179.00,-4458211.00,45114.00,-677421.00,45114.00,426954.00,55770.00,320536.10,29718.00,5429

In [76]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.62


In [77]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[6224 1357]
 [3772 1971]]


In [78]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.82      0.71      7581
           1       0.59      0.34      0.43      5743

    accuracy                           0.62     13324
   macro avg       0.61      0.58      0.57     13324
weighted avg       0.61      0.62      0.59     13324

